## Źródło danych

W projekcie oparliśmy się o zbiór danych pobrany z serwisu Kaggle.com. Zbiór danych opisuje 100 tys. książek z serwisu Goodreads.com. Więcej na temat zbioru danych można przeczytać pod linkiem: https://www.kaggle.com/datasets/mdhamani/goodreads-books-100k

Z Wikipedii:
> Kaggle to platforma konkursowa data science i internetowa społeczność naukowców zajmujących się danymi i praktyków uczenia maszynowego w ramach Google LLC.

## Import zależności

Wykorzystujemy pakiety:
- requests – do pobierania plików ze skanami okładek
- os – do operacji na plikach
- colorthief – do ustalenia domunującego koloru na okładce
- sklearn.neighbors – do dopasowania najbliższego nazwanego koloru
- numpy – do operacji na tablicach
- pandas – do operacji na ramkach danych


In [5]:
import requests
import os
from colorthief import ColorThief
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

## Definicje funkcji

W tym miejscu definiujemy pomocnicze funkcje użyte w zasadniczej części kodu (w następnych sekcjach).

Kolor w formacie RGB może być określony bardzo dokładnie. Funkcja `ColorThief.get_color()` może zwrócić (2^8)^3 = 16 777 216 różnych kolorów dlatego posłużyliśmy się mapą kolorów CSS (https://www.quackit.com/css/css_color_codes.cfm), aby sprowadzić je do 11 kolorów głównych. Percepcja koloru jest rzeczą subiektywną, zdajemy sobie sprawę, że jeden kolor przez różne osoby może być nazwany różnie, na dodatek wiele okładek jest różnokolorowych. Program ustala kolor w sposób mocno obiektywny najpierw znajdując kolor dominujący, a następnie dopasowując ten kolor za pomocą modelu uczenia maszynowego NearestNeighbors do najbliższego koloru z 11 głównych przy pomocy tabeli.

In [6]:

# Pobiera plik okładki z podanego URL
def down(url, index):
  response = requests.get(url)
  img_data = response.content
  with open('../tmp/' + str(index), 'wb') as handler:
    handler.write(img_data)

# Pobiera dominujący kolor w formacie RGB z pliku
def get_dominant_color(index):
  color_thief = ColorThief('../tmp/' + str(index))
  dominant_color = color_thief.get_color(quality=1)
  return dominant_color

# Usuwa plik o podanym indeksie
def remove(index):
   os.remove(index)

# Lista kolorów głównych na podstawie tabeli https://www.quackit.com/css/css_color_codes.cfm
known_colors = {
  "red": (205,92,92),
  "red": (240,128,128),
  "red": (250,128,114),
  "red": (233,150,122),
  "red": (255,160,122),
  "red": (220,20,60),
  "red": (255,0,0),
  "red": (178,34,34),
  "red": (139,0,0),
  "pink": (255,192,203),
  "pink": (255,182,193),
  "pink": (255,105,180),
  "pink": (255,20,147),
  "pink": (199,21,133),
  "pink": (219,112,147),
  "orange": (255,127,80),
  "orange": (255,99,71),
  "orange": (255,69,0),
  "orange": (255,140,0),
  "orange": (255,165,0),
  "yellow": (255,215,0),
  "yellow": (255,255,0),
  "yellow": (255,255,224),
  "yellow": (255,250,205),
  "yellow": (250,250,210),
  "yellow": (255,239,213),
  "yellow": (255,228,181),
  "yellow": (255,218,185),
  "yellow": (238,232,170),
  "yellow": (240,230,140),
  "yellow": (189,183,107),
  "purple": (230,230,250),
  "purple": (216,191,216),
  "purple": (221,160,221),
  "purple": (238,130,238),
  "purple": (218,112,214),
  "purple": (255,0,255),
  "purple": (255,0,255),
  "purple": (186,85,211),
  "purple": (147,112,219),
  "purple": (138,43,226),
  "purple": (148,0,211),
  "purple": (153,50,204),
  "purple": (139,0,139),
  "purple": (128,0,128),
  "purple": (102,51,153),
  "purple": (75,0,130),
  "purple": (123,104,238),
  "purple": (106,90,205),
  "purple": (72,61,139),
  "green": (173,255,47),
  "green": (127,255,0),
  "green": (124,252,0),
  "green": (0,255,0),
  "green": (50,205,50),
  "green": (152,251,152),
  "green": (144,238,144),
  "green": (0,250,154),
  "green": (0,255,127),
  "green": (60,179,113),
  "green": (46,139,87),
  "green": (34,139,34),
  "green": (0,128,0),
  "green": (0,100,0),
  "green": (154,205,50),
  "green": (107,142,35),
  "green": (128,128,0),
  "green": (85,107,47),
  "green": (102,205,170),
  "green": (143,188,143),
  "green": (32,178,170),
  "green": (0,139,139),
  "green": (0,128,128),
  "blue": (0,255,255),
  "blue": (0,255,255),
  "blue": (224,255,255),
  "blue": (175,238,238),
  "blue": (127,255,212),
  "blue": (64,224,208),
  "blue": (72,209,204),
  "blue": (0,206,209),
  "blue": (95,158,160),
  "blue": (70,130,180),
  "blue": (176,196,222),
  "blue": (176,224,230),
  "blue": (173,216,230),
  "blue": (135,206,235),
  "blue": (135,206,250),
  "blue": (0,191,255),
  "blue": (30,144,255),
  "blue": (100,149,237),
  "blue": (65,105,225),
  "blue": (0,0,255),
  "blue": (0,0,205),
  "blue": (0,0,139),
  "blue": (0,0,128),
  "blue": (25,25,112),
  "brown": (255,248,220),
  "brown": (255,235,205),
  "brown": (255,228,196),
  "brown": (255,222,173),
  "brown": (245,222,179),
  "brown": (222,184,135),
  "brown": (210,180,140),
  "brown": (188,143,143),
  "brown": (244,164,96),
  "brown": (218,165,32),
  "brown": (184,134,11),
  "brown": (205,133,63),
  "brown": (210,105,30),
  "brown": (139,69,19),
  "brown": (160,82,45),
  "brown": (165,42,42),
  "brown": (128,0,0),
  "white": (255,255,255),
  "white": (255,250,250),
  "white": (240,255,240),
  "white": (245,255,250),
  "white": (240,255,255),
  "white": (240,248,255),
  "white": (248,248,255),
  "white": (245,245,245),
  "white": (255,245,238),
  "white": (245,245,220),
  "white": (253,245,230),
  "white": (255,250,240),
  "white": (255,255,240),
  "white": (250,235,215),
  "white": (250,240,230),
  "white": (255,240,245),
  "white": (255,228,225),
  "gray": (220,220,220),
  "gray": (211,211,211),
  "gray": (211,211,211),
  "gray": (192,192,192),
  "gray": (169,169,169),
  "gray": (169,169,169),
  "gray": (128,128,128),
  "gray": (128,128,128),
  "gray": (105,105,105),
  "gray": (105,105,105),
  "gray": (119,136,153),
  "gray": (119,136,153),
  "gray": (112,128,144),
  "gray": (112,128,144),
  "gray": (47,79,79),
  "gray": (47,79,79),
  "black": (0,0,0)
}

# Przygotowanie danych dla NearestNeighbors
color_names = list(known_colors.keys())
color_values = np.array(list(known_colors.values()))

# Utworzenie modelu NearestNeighbors (model uczenia maszynowego nienadzorowanego)
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
model = NearestNeighbors(n_neighbors=1).fit(color_values)

# Funkcja zwracająca nazwę koloru na podstawie RGB
def get_color_name(rgb):
    # Use the NearestNeighbors model to find the closest known color
    distance, index = model.kneighbors(np.array([rgb]))
    return color_names[index[0][0]]

## Wczytanie ramki danych

Program przetwarzający okładki ma ograniczoną wydajność, ponieważ musi każdą okładkę osobno pobrać, zapisać na dysku, pobrać z niej kolor domunujący, znaleźć kolor główny, zaktualizować ramkę danych, usunąć plik okładki (pliki są usuwane od razy ponieważ wszystkie zajęłyby wiele GB, a na GitHub Codespaces powierzchnia dyskowa jest ograniczona).

Przetwarzanie osiąga prędkość około 100 okładek na minutę. Przetworzenie 100 tys. okładek zajmuje kilkanaście godzin.

Co 100 rekordów skrypt wykonuje aktualizację pliku CSV z danymi na dysku, aby uniknąć utraty wyników w przypadku przerwania programu. Dlatego skrypt ładujący ramkę danych potrafi rozpoznać, czy istnieje już plik z wynikami, jeśli tak ładuje plik z częściowo wykonaną pracą, a jeśli nie zakłada nowy plik z wynikami.

Podczas wczytywania danych pomijane są książki bez okładek. 96 955 książek ma okładkę, czyli dokładnie 96,955%.

In [7]:
if (os.path.isfile('../data/books.csv')):
  books = pd.read_csv('../data/books.csv', keep_default_na=False)
else:
  books = pd.read_csv('../data/orig/GoodReads_100k_books.csv', keep_default_na=False)
  
  # Usuń wiersze z brakującym obrazkiem
  books = books[books['img'] != '']
  
  books['dominant_color'] = ''
  books['dominant_color_name'] = ''
  
  books.to_csv('../data/books.csv', index=False)

books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96955 entries, 0 to 96954
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   author               96955 non-null  object 
 1   bookformat           96955 non-null  object 
 2   desc                 96955 non-null  object 
 3   genre                96955 non-null  object 
 4   img                  96955 non-null  object 
 5   isbn                 96955 non-null  object 
 6   isbn13               96955 non-null  object 
 7   link                 96955 non-null  object 
 8   pages                96955 non-null  int64  
 9   rating               96955 non-null  float64
 10  reviews              96955 non-null  int64  
 11  title                96955 non-null  object 
 12  totalratings         96955 non-null  int64  
 13  dominant_color       96955 non-null  object 
 14  dominant_color_name  96955 non-null  object 
dtypes: float64(1), int64(3), object(11)


## Przetwarzanie danych

W tym miejscu znajduje się główna pętla programu która iteruje po książkach. Jeśli napotka brak koloru w ramce danych wykonuje odpowiednie kroki.

Obsługa błędów za pomocą bloków `try ... except` zapobiega przerwaniu programu w przypadku błędów (sporadycznie pojawia się błąd `Empty pixels when quantize.`, który może wynikać z uszkodzenia pliku z okładką lub błędu w samym pakiecie `ColorThief`).

In [ ]:
counter = 0
for index, row in books.iterrows():
    if not row['dominant_color_name']:
      try:
        counter += 1

        down(row['img'], index)

        rgb = get_dominant_color(index)
        books.at[index, 'dominant_color'] = rgb
        books.at[index, 'dominant_color_name'] = get_color_name(rgb)

        os.remove('../tmp/' + str(index))

        if (0 == counter % 100):
          books.to_csv('../data/books.csv', index=False)
          print("Processed {} rows, currently on index {}".format(counter, index))

      except Exception as e:
         print("Exception: {} (Index: {})".format(str(e), index))

books.to_csv('../data/books.csv', index=False)
print("Processed {} rows, currently on index {}".format(counter, index))

In [10]:
books['dominant_color_name'].value_counts()

gray      23877
yellow    21139
white     20595
black     10709
purple     5414
pink       4734
red        3286
green      2149
orange     2104
brown      1563
blue       1142
            243
Name: dominant_color_name, dtype: int64